In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from pathlib import Path

In [3]:
abc = !pwd
isLocal = (abc[0].find('anooshhari') != 0)

In [4]:
if isLocal:
    base_directory = "/Users/anooshhari/Documents/Anoosh/Masters/Spring 21/Classes/6100 - BigData Analytics for C/Project/indoor-location-navigation"
    base_git = base_directory
    test_directory = "/Users/anooshhari/Documents/Anoosh/Masters/Spring 21/Classes/6100 - BigData Analytics for C/Project/indoor-location-navigation/test"
else:
    base_directory = "/kaggle/input/indoor-location-navigation/"
    base_git = "/kaggle/working/"

In [5]:
if not isLocal:
    !git clone https://github.com/location-competition/indoor-location-competition-20.git

In [6]:
import sys
sys.path.append(base_git +'/indoor-location-competition-20')
from io_f import read_data_file

In [7]:
import glob

txtfiles = []
for file in glob.glob(f"{base_directory}/train/*/*/*.txt"):
    lst = file.split("/")[-3:]
    lst.append(file)
    txtfiles.append(lst)

In [8]:
df = pd.DataFrame(txtfiles,columns=['site_id','floor','path_id',"file_path"])
df["file_data"] = None

In [9]:
site1_b1 = df[df.site_id.eq('5a0546857ecc773753327266')]

In [10]:
site1_b1.head()

,site_id,floor,path_id,file_path,file_data
0,5a0546857ecc773753327266,F4,5d79be0d43d77a0006b63a6d.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None
1,5a0546857ecc773753327266,F4,5d79be0043d77a0006b63a5f.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None
2,5a0546857ecc773753327266,F4,5d11dc04ffe23f0008604f57.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None
3,5a0546857ecc773753327266,F4,5d8f0a4db6e29d0006fb8c36.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None
4,5a0546857ecc773753327266,F4,5d8f0a52d5bae80006eb8de3.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None


In [11]:
def remove_timestamp_column(data):
    for k,v in data.items():
        if k in ['acce','gyro','magn','ahrs']:
            data[k] = v[:,1:]
    return data

In [12]:
# read_data_file(site1_b1.file_path.values[0])
for i,row in site1_b1.iterrows():
    readData = read_data_file(row.file_path)
    row.file_data = pd.DataFrame.from_dict({k:list(v) for k,v in remove_timestamp_column(readData.__dict__).items()}, orient='index', dtype=object).transpose()

In [ ]:
# pd.DataFrame.from_dict({k:list(v) for k,v in read_data_file(site1_b1.file_path.values[0]).__dict__.items()}, orient='index').transpose()

In [15]:
list(site1_b1.head().floor.values)

['F4', 'F4', 'F4', 'F4', 'F4']

In [ ]:
site1_b1.file_data.values[0]['acce'].values[0][:,1:]

In [ ]:
from main import calibrate_magnetic_wifi_ibeacon_to_position

In [ ]:
site1_b1.file_path[0:1].values[0]

In [ ]:
single_path_data = calibrate_magnetic_wifi_ibeacon_to_position(site1_b1.file_path[0:1])

In [ ]:
len(single_path_data.keys())

In [ ]:
list(site1_b1.file_data.values[0].acce.values[0][:,0])

In [ ]:
site1_b1.file_data.values[0][['acce','gyro','magn','ahrs']].shape

In [ ]:
site1_b1.file_data.values[0][['acce','gyro','magn','ahrs']].dropna().shape

In [13]:
data_combined = pd.DataFrame(columns=['acce','gyro','magn','ahrs','floor'])
print(data_combined)
for i,data in site1_b1.iterrows():
    current = data.file_data[['acce','gyro','magn','ahrs']].dropna()
    current['floor'] = data.floor
    current.reset_index(drop=True)
#     print(type(data))
#     print(type(current))
#     break
    data_combined = pd.concat([data_combined, current])
# print(data_combined)

Empty DataFrame
Columns: [acce, gyro, magn, ahrs, floor]
Index: []


In [ ]:
# data_combined.describe()

In [14]:
data_combined.head()

,acce,gyro,magn,ahrs,floor
0,"[-1.4918823, 0.34014893, 10.652069]","[0.2646942, -0.29737854, 0.27461243]","[-23.14148, -18.16864, -23.690796]","[0.06200746, 0.026087942, -0.87248856]",F4
1,"[-1.5619049, 0.4065857, 10.006226]","[0.2641449, -0.27552795, 0.22401428]","[-25.195312, -19.537354, -25.686646]","[0.06252557, 0.021754283, -0.8709258]",F4
2,"[-1.6876068, 0.5298767, 9.641113]","[0.2055664, -0.17752075, 0.21813965]","[-25.195312, -17.48352, -25.686646]","[0.067413874, 0.016335025, -0.86725426]",F4
3,"[-1.6343384, 0.6310425, 9.501663]","[0.15975952, -0.05343628, 0.24797058]","[-23.14148, -17.48352, -23.690796]","[0.067413874, 0.016335025, -0.86725426]",F4
4,"[-1.5900421, 0.7214203, 9.437012]","[0.17041016, 0.004638672, 0.25810242]","[-23.14148, -19.537354, -24.35608]","[0.07057936, 0.014525489, -0.8652526]",F4


In [15]:
list(data_combined.floor.value_counts().keys())

['F2', 'F4', 'B1', 'F1', 'F3']

In [ ]:
data_combined_copy = data_combined.copy()
data_combined_copy.replace({"floor":{'F2':1, 'F4':3, 'B1':-1, 'F1':0, 'F3':2}}, inplace=True)

In [16]:
def floor_to_numeric(floor):
    if 'F' in floor:
        return int(floor.replace('F',''))-1
    elif 'B' in floor:
        return (int(floor.replace('B','')))*(-1)

0    0
1    0
2    1
3    1
4   -1
5   -1
6   -2
7   -2
dtype: int64

In [ ]:
data_combined_copy.head()

In [ ]:
# data_combined_copy = pd.get_dummies(data_combined, columns=["floor"], prefix=["floor"])

In [ ]:
data_combined_copy.acce = data_combined_copy.acce.apply(lambda x: x/np.sqrt((x**2).sum()))
data_combined_copy.gyro = data_combined_copy.gyro.apply(lambda x: x/np.sqrt((x**2).sum()))
data_combined_copy.magn = data_combined_copy.magn.apply(lambda x: x/np.sqrt((x**2).sum()))
data_combined_copy.ahrs = data_combined_copy.ahrs.apply(lambda x: x/np.sqrt((x**2).sum()))
# data_combined_copy[['acce','gyro','magn','ahrs']] = data_combined_copy[['acce','gyro','magn','ahrs']].apply(lambda x: x/np.sqrt((x**2).sum()))

In [ ]:
data_combined_copy.shape

In [ ]:
# !pip install keras
# !pip install tensorflow
# !pip install layers

In [ ]:
import keras
from keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
train, test = train_test_split(data_combined_copy, test_size=0.4)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
np.random.uniform(0,1, 3)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((train[['acce','gyro','magn','ahrs']].values, train.floor.values))

In [ ]:
data_combined_junk = data_combined_copy.copy()

In [ ]:
data_combined_junk[['acce_x','acce_y','acce_z']] = pd.DataFrame(data_combined_junk.acce.tolist(), index= data_combined_junk.index)

In [ ]:
data_combined_junk[['gyro_x','gyro_y','gyro_z']] = pd.DataFrame(data_combined_junk.gyro.tolist(), index= data_combined_junk.index)

In [ ]:
data_combined_junk[['magn_x','magn_y','magn_z']] = pd.DataFrame(data_combined_junk.magn.tolist(), index= data_combined_junk.index)

In [ ]:
data_combined_junk[['ahrs_x','ahrs_y','ahrs_z']] = pd.DataFrame(data_combined_junk.ahrs.tolist(), index= data_combined_junk.index)

In [ ]:
data_combined_junk.head()

In [ ]:
data_combined_split = data_combined_junk[data_combined_junk.columns[4:]].copy()

In [ ]:
data_combined_split.head()

In [ ]:
train, test = train_test_split(data_combined_split, test_size=0.4)

In [ ]:
def getmodel():
    
    x_input = keras.Input(shape=(None, train.shape[0], train.shape[1]-1),name="input")  # Variable-length sequence of ints
    conv1d_1= layers.Conv1D(64, 1, activation='relu')(x_input)
    flat = layers.Flatten()(conv1d_1)
    dense1 = layers.Dense(50, name="dense1", activation="relu")(flat)
    encoder_conv_dense2 = layers.Dense(50, name="40_dense", activation="relu")(dense1)
    encoder_conv_dense3 = layers.Dense(1, name="floor", activation="softmax")(encoder_conv_dense2)

    model = keras.Model(
        inputs=x_input,
        outputs=encoder_conv_dense3,
    )
    
    return model

In [ ]:
train[train.columns[1:]]

In [ ]:
train.floor = train.floor+1
dataset = tf.data.Dataset.from_tensor_slices((train[train.columns[1:]].values, train['floor'].values))

In [ ]:
train_dataset = dataset.shuffle(len(dataset)).batch(1)

In [ ]:
def get_compiled_model():
  model_new = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
  ])

  model_new.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])
  return model_new

In [ ]:
model_new = get_compiled_model()
model_new.fit(train_dataset, epochs=3)

In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices(test[test.columns[1:]].values)

In [ ]:
test_predictions = model_new.predict(test[test.columns.values[1:]])

In [ ]:
final_predictions = np.argmax(test_predictions, axis=1)-1

In [ ]:
test.floor.values

In [ ]:
final_predictions

In [ ]:
values = test.floor.values == final_predictions

In [ ]:
values.sum()

In [ ]:
values.size

In [ ]:
sample_csv = pd.read_csv('/Users/anooshhari/Documents/Anoosh/Masters/Spring 21/Classes/6100 - BigData Analytics for C/Project/indoor-location-navigation/sample_submission.csv')

In [ ]:
sample_csv.head()

In [ ]:
sample_csv['site']=None
sample_csv['path']=None

In [ ]:
submission_data = sample_csv.copy()
for i, row in submission_data.iterrows():
#     print(row.site_path_timestamp.split("_"))
    row.site.loc(i) = row.site_path_timestamp.split("_")[0]
#     print(row.site)
#     row['path'], row['site'] = row.site_path_timestamp.split('_')[1], row.site_path_timestamp.split('_')[0]

In [ ]:
submission_data = pd.DataFrame(sample_csv)

In [ ]:
# for i,row in submission_data.iterrows():
#     row['site'] = row.site_path_timestamp.split("_")[0]
#     row['path'] = row.site_path_timestamp.split("_")[1]
#     print(row)
submission_data.site = submission_data.site_path_timestamp.apply(lambda x: x.split("_")[0])

In [ ]:
submission_data.path = submission_data.site_path_timestamp.apply(lambda x: x.split("_")[1]+".txt")

In [ ]:
submission_data.shape

In [ ]:
submission_data['model_input']=None
submission_data[submission_data.site == '5a0546857ecc773753327266']

In [ ]:
def transform_file_to_dataframe(file_names):
    file_names = list(set(file_names))
    input_data = pd.DataFrame(columns=['file_name', 'file_data'])
    input_data.file_name = file_names
    for i, row in input_data.iterrows():
        readData = read_data_file(test_directory+"/"+row.file_name)
        row.file_data = pd.DataFrame.from_dict({k:list(v) \
                                                for k,v in remove_timestamp_column(readData.__dict__).items()},\
                                                orient='index', dtype=object).transpose()
#     print(input_data)
    
    data_combined = pd.DataFrame(columns=['acce','gyro','magn','ahrs'])
#     print("data combined empty",data_combined)
    for i,data in input_data.iterrows():
        current = data.file_data[['acce','gyro','magn','ahrs']].dropna()
        current.reset_index(drop=True)
        data_combined = pd.concat([data_combined, current])
        
#     print("data combined full",data_combined)
    
    data_combined.acce = data_combined.acce.apply(lambda x: x/np.sqrt((x**2).sum()))
    data_combined.gyro = data_combined.gyro.apply(lambda x: x/np.sqrt((x**2).sum()))
    data_combined.magn = data_combined.magn.apply(lambda x: x/np.sqrt((x**2).sum()))
    data_combined.ahrs = data_combined.ahrs.apply(lambda x: x/np.sqrt((x**2).sum()))
    
#     print("after normalization", data_combined)
    
    data_combined[['acce_x','acce_y','acce_z']] = pd.DataFrame\
    (data_combined.acce.tolist(), index= data_combined.index)
    
    data_combined[['gyro_x','gyro_y','gyro_z']] = pd.DataFrame\
    (data_combined.acce.tolist(), index= data_combined.index)
        
    data_combined[['magn_x','magn_y','magn_z']] = pd.DataFrame\
    (data_combined.acce.tolist(), index= data_combined.index)
    
    data_combined[['ahrs_x','ahrs_y','ahrs_z']] = pd.DataFrame\
    (data_combined.acce.tolist(), index= data_combined.index)
    
    data_combined = data_combined[['acce_x','acce_y','acce_z',\
                                   'gyro_x','gyro_y','gyro_z',\
                                   'magn_x','magn_y','magn_z',\
                                   'ahrs_x','ahrs_y','ahrs_z']]
    return data_combined

In [ ]:
output_onefile = transform_file_to_dataframe(['046cfa46be49fc10834815c6.txt'])
# testing123 = pd.DataFrame(columns=['file_name', 'file_data'])

In [ ]:
output_onefile

In [ ]:
predictions = model_new.predict(output_onefile)

In [ ]:
predictions

In [ ]:
pd.Series(np.argmax(predictions, axis=1)-1).value_counts()[0:1].keys().values[0]

In [ ]:
# for i, row in submission_data[submission_data.site == '5a0546857ecc773753327266'].iterrows():
#     row.model_input = transform_file_to_dataframe([row.path])

In [ ]:
# for i, row in submission_data[submission_data.site == '5a0546857ecc773753327266'].iterrows():
#     abc = dict()
#     if row.path in abc.keys():
#         row.model_input = abc[row.path]
#     else:
#         row.model_input = transform_file_to_dataframe([row.path])
#         abc.add(row.path, row.model_input)
abc = dict()
def model_inputs(val):
    if val in abc.keys():
        return abc[val]
    else:
        res = transform_file_to_dataframe([val])
        abc[val] = res
        return res

In [ ]:
submission_data[submission_data.site == '5a0546857ecc773753327266'].model_inputs = submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x))

In [ ]:
# submission_data[submission_data.site == '5a0546857ecc773753327266']
# abc[1]=123
# len(list(abc.keys()))
# len(set(list(submission_data.path[submission_data.site=='5a0546857ecc773753327266'].values)))

In [ ]:
submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x)).values

In [ ]:
submission_data[submission_data.site == '5a0546857ecc773753327266'].model_input = submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x)).values

In [ ]:
submission_data.loc[submission_data.site.str.startswith('5a0546857ecc773753327266'), 'model_input'] = submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x))

In [ ]:
submission_data[submission_data.site == '5a0546857ecc773753327266']

In [ ]:
prediction_floor = dict()
def final_floor_predictions_submission(df):
    for i,row in df.iterrows():
        if row.path in prediction_floor.keys():
            row['floor'] = prediction_floor[row.path]
        else:
            predicts = model_new.predict(row.model_input)
            floor_value = pd.Series(np.argmax(predicts, axis=1)-1).value_counts()[0:1].keys().values[0]
            prediction_floor[row.path] = floor_value
            row['floor'] = floor_value
        print(row.floor)

In [ ]:
final_floor_predictions_submission(submission_data[submission_data.site == '5a0546857ecc773753327266'])

In [ ]:
submission_data[submission_data.site == '5a0546857ecc773753327266'].loc[:,'floor'] = 3

In [ ]:
submission_data.loc[submission_data.site.str.startswith('5a0546857ecc773753327266'), 'floor']=3

In [ ]:
submission_data[submission_data.site == '5a0546857ecc773753327266']

In [ ]:
submission_data.iloc[:,0:4].to_csv('submission.csv', index=False)

In [ ]:
# for data in list(abc.values()):
#     for i,row in data.iterrows():
        
model_new.predict(list(abc.values())[9])

In [ ]:
np.argmax(test_predictions, axis=1)